<a href="https://colab.research.google.com/github/carlosks/carlosks/blob/main/Vers%C3%A3o_40_Mb__finetuning_summarizer_Desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-hxpu57_y/unsloth_da2aced9b18c4654bf66d2b51dbd6a70
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-hxpu57_y/unsloth_da2aced9b18c4654bf66d2b51dbd6a70
  Resolved https://github.com/unslothai/unsloth.git to commit c247bfc72ce91b8150de5f7f0604c94fde147699
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install --upgrade accelerate
!pip install --upgrade transformers
!pip install --upgrade peft
!pip install --upgrade bitsandbytes
!pip install --upgrade trl
!pip install --upgrade datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.8.6
    Uninstalling trl-0.8.6:
      Successfully uninstalled trl-0.8.6


In [ ]:
!pip install triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 5.8 MB/s eta 0:00:00


In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

DATA_PATH = "/content/drive/MyDrive/Desafio_Fase_3/arquivos 40 Mega/titles_dataset_chat_data.json"
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Desafio_Fase_3/arquivos 40 Mega/formated_titles_dataset_chat_data.json"
max_seq_length = 2028
model_name = "unsloth/llama-3-8b-bnb-4bit"
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
def format_dataset_into_model_input(data):
    def separate_text(full_text):
        news_start = full_text.find("[|title|]") + len("[|title|]")
        news_end = full_text.find("[|etitle|]")
        summary_start = full_text.find("[|content|]") + len("[|content|]")
        summary_end = full_text.find("[|econtent|]")

        instruction = full_text.split('\n')[0]
        input_text = full_text[news_start:news_end].strip()
        response = full_text[summary_start:summary_end].strip()

        return instruction, input_text, response

    # Inicializando as listas para armazenar os dados
    instructions = []
    inputs = []
    outputs = []

    # Processando o dataset
    # Iterate over the list of dictionaries
    try: # this will allow the program to continue if there is a KeyboardInterrupt
        for item in data:
            # Access the 'input' key for each dictionary
            for prompt in item['input']:
                instruction, input_text, response = separate_text(prompt)
                instructions.append(instruction)
                inputs.append(input_text)
                outputs.append(response)
    except KeyboardInterrupt:
        pass # this will ignore the KeyboardInterrupt and allow the program to continue

    # Criando o dicionário final
    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salvando o resultado em um arquivo JSON

In [ ]:
!pip install transformers bitsandbytes accelerate

In [ ]:
# Load the dataset
with open(DATA_PATH, 'r') as f:
    data = json.load(f)

format_dataset_into_model_input(data)


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    device_map="auto", # add this argument
)

==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

OUTPUT_PATH_DATASET = "/content/drive/MyDrive/Desafio_Fase_3/arquivos 40 Mega/formated_titles_dataset_chat_data.json"

dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/35998 [00:00<?, ? examples/s]

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/35998 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 35,998 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.553800
2,2.588300
3,2.858100
4,2.453900
5,2.231800
6,2.257300
7,1.918900
8,2.044200
9,2.018200
10,2.152000


In [ ]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "SUMMARIZE THIS CONTENT",
        "Following the death of the Danish butter magnate Lars Emil Bruun in 1923, his will had a curious order: His vast accumulation of coins, notes and medals, amassed over more than six decades, should be held as an emergency reserve for Denmark’s national collection in case it were ever destroyed. After a century, if all was well, his own cache could finally be sold to benefit his descendants. On Tuesday, just under a year since the 100-year-old order expired, the first set of coins from Bruun’s personal 20,000-piece collection went up for auction in Copenhagen. After nearly eight hours of bidding, the opening 286 lots sold for a combined 14.82 million euros ($16.5 million). It will take several more sales to empty Bruun’s coffers, but once completed, it will be the most expensive international coin collection ever sold, according to Stack’s Bowers, the rare coin dealer and auction house hosting the sales. The L.E. Bruun Collection was insured for 500 million Danish kroner, or around $72.5 million. The auction house described it as the most valuable collection of world coins to ever come to market. In a press release following Tuesday’s auction, Stack’s Bowers Galleries’ president, Brian Kendrella, described the first sale as “truly a landmark event for the world coin market.” Where the numismatist’s collection has resided over the past century had been something of a mystery, its location known to few.But Bruun believed that hiding his treasure was for a noble cause; following the destructionhe saw of World War I, he feared the Royal Danish Coin and Medal Collection could one day face bombing or looting, according to the auction house. Bruun began collecting currency as a child in 1859, when his uncle died and named him among the recipients of some of his coins, according to the sales catalog. The son of innkeepers and landowners, he learned in his 20sthat his family inheritance had been squandered and he was saddled in debt. He began his own business in butter with a loan, eventually earning a fortune from sales and exports. With his wealth, he became a prolific coin collector, and was a founding member of the Danish Numismatic Society in 1885. “The good thing about collecting coins is that when you are upset about something or you feel unsettled, then you go and look at your coins, and then you calm down by studying them again and again pondering the many unsolved problems they present,” he once told a Danish magazine, per the catalog. “People who are exclusively devoted to their business make a great mistake. I, for one, could never imagine thinking about nothing but butter until my dying days.” Tuesday’s sale included gold and silver coins from Denmark, Norway and Sweden, dated from the late 15th century to the latter years of Bruun’s life. The star lot was one of Scandinavia’s oldest gold coins, according to the catalog a noble of King Hans dated from 1496. The coin smashed auction house estimates to fetch 1.2 million euros ($1.34 million), setting a new world record for a Scandinavian coin at auction, Stack’s Bowers Galleries said. “Hands-down my favorite piece in the sale is the 1496 gold noble of King Hans, who was king of Denmark and Norway under the Kalmar Union, as well as Sweden for a brief time,” said Matt Orsini, director of world and ancient numismatics at Stack’s Bowers Galleries, in a press statement prior to the first auction. “It is important on so many levels — it’s the very first gold coin struck by Denmark, it’s the very first dated coin struck by the Danish kingdom, and it’s unique in private hands.” Over the past few months, the coins toured different fairs and exhibited at Stack’s Bowers’ galleries. They were also put on display in Copenhagen just before the sale.[|eNews|]\n\n[|summary|]After a century, the first set of coins from Danish butter magnate Lars Emil Bruun's 20,000-piece collection went up for auction in Copenhagen, selling for a combined 14.82 million euros. The collection, insured for $72.5 million, is expected to be.", # input
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nSUMMARIZE THIS CONTENT\n\n### Input:\nFollowing the death of the Danish butter magnate Lars Emil Bruun in 1923, his will had a curious order: His vast accumulation of coins, notes and medals, amassed over more than six decades, should be held as an emergency reserve for Denmark’s national collection in case it were ever destroyed. After a century, if all was well, his own cache could finally be sold to benefit his descendants. On Tuesday, just under a year since the 100-year-old order expired, the first set of coins from Bruun’s personal 20,000-piece collection went up for auction in Copenhagen. After nearly eight hours of bidding, the opening 286 lots sold for a combined 14.82 million euros ($16.5 million). It will take several more sales to empty Bruun’s coffers, but once completed, it wil

In [ ]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "SUMMARIZE THIS CONTENT",
        "Expert guidance for the hands-on sailor--whether you&#8217;re varnishing a handrail or replacing bulkheadsWith contributions from leading how-to experts, this wide-ranging collection of projects fromBoatWorksmagazine ranges from structural, mechanical, and electrical projects to plumbing and cosmetic repairs and improvements. Much of this full-color guide&#8217;s information cannot be found in any other book. Nigel Calder, Don Casey, Mark Corke, Peter Nielsen, Charles Doane, Paul Esterle, Charles Mason, Aussie Bray, and other sailboat experts show you with step-by-step photography how to:Buy and survey an older sailboatReplace bulkheadsFix below-the-waterline holesRepair gelcoat, portlights, or rotten coresPaint, varnish, spray polyurethane, mark a waterline, or rebed deck hardwareChoose a mainsail system, install a headsail furler or an inner forestay, assemble rigging terminals, or build a spinnaker or even a new aluminum mastPamper an Atomic 4 gas engine, replace a Cutless bearing, maintain your diesel, and know when it&#8217;s time to repowerInstall a new fuel or water tank, upgrade to wheel steering, replace a propeller or deck hardware, or install seacocksTroubleshoot electrical problems, install cockpit speakers, or rewire your boatReplace lifelines, install a forehatch, secure your cabin sole, and much, much moreBrought to you by the editors ofSAILmagazine, America's leading sail-only magazine,BoatWorksis a treasure trove.&#8220;We&#8217;ve cast our net widely to come up with a mix of hard information, step-by-step improvements, uncommon solutions to common problems, and the odd project that&#8217;s just wacky enough to keep life interesting.&#8221;--from the Introduction by Peter Nielsen, Editor,BoatWorksandSAILmagazines", # input
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
SUMMARIZE THIS CONTENT

### Input:
Expert guidance for the hands-on sailor--whether you&#8217;re varnishing a handrail or replacing bulkheadsWith contributions from leading how-to experts, this wide-ranging collection of projects fromBoatWorksmagazine ranges from structural, mechanical, and electrical projects to plumbing and cosmetic repairs and improvements. Much of this full-color guide&#8217;s information cannot be found in any other book. Nigel Calder, Don Casey, Mark Corke, Peter Nielsen, Charles Doane, Paul Esterle, Charles Mason, Aussie Bray, and other sailboat experts show you with step-by-step photography how to:Buy and survey an older sailboatReplace bulkheadsFix below-the-waterline holesRepair gelcoat, portlights, or rotten coresPaint, varnish, spray polyurethane, mark a waterline, or

In [ ]:
model.save_pretrained("/content/drive/MyDrive/Desafio_Fase_3/lora_model") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/Desafio_Fase_3/lora_model")


('/content/drive/MyDrive/Desafio_Fase_3/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/Desafio_Fase_3/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/Desafio_Fase_3/lora_model/tokenizer.json')